In [32]:
import numpy as np
import pandas as pd
import jieba
stop_word = pd.read_csv('停用词.csv',names=['w'],sep='\t',encoding='utf-8')
view=pd.read_csv('景点-1546.csv')
view=view.dropna().reset_index().drop('index',axis=1)

In [ ]:
#这样分词会导致word2vec形成的都是单字
for i in range(len(view.txt)):
    view.txt[i]=(' '.join([w for w in jieba.cut(view.txt[i]) if w not in list(stop_word.w)]))
view

In [55]:
#这样可以成功
#view['cut'] = view.txt.apply(jieba.lcut)
for i in range(len(view.txt)):
    view.cut[i] = [w for w in jieba.cut(view.txt[i]) if w not in list(stop_word.w)]
view.head()

,spot,txt,cut
0,秦始皇帝陵博物院（兵马俑）,乐乐服务周到非常贴心兵马俑震撼人心,"[乐乐, 服务周到, 贴心, 兵马俑, 震撼人心]"
1,秦始皇帝陵博物院（兵马俑）,服务很好讲解深刻到位人很好秦始皇兵马俑介绍非常详细乐乐棒棒哒,"[服务, 深刻, 到位, 秦始皇, 兵马俑, 介绍, 详细, 乐乐, 棒棒, 哒]"
2,秦始皇帝陵博物院（兵马俑）,乐乐热情大方旅途很愉快开心开心开心,"[乐乐, 热情, 大方, 旅途, 愉快, 开心, 开心, 开心]"
3,秦始皇帝陵博物院（兵马俑）,可以可以可以可以可以可以,[]
4,秦始皇帝陵博物院（兵马俑）,直通车很方便对乐乐的服务很满意,"[直通车, 乐乐, 服务, 满意]"


In [56]:
from gensim.models.word2vec import Word2Vec

n_dim = 300 # 指定向量维度，大样本量时300~500较好

w2vmodel = Word2Vec(vector_size = n_dim, min_count = 10) # size改为vector_size
w2vmodel.build_vocab(view.cut) # 生成词表
w2vmodel

In [57]:
'西安' in w2vmodel.wv.index_to_key

True

In [58]:
# 在评论训练集上建模（大数据集时可能会花费几分钟）
# 本例消耗内存较少 100次 40s
w2vmodel.train(view.txt,total_examples = w2vmodel.corpus_count, epochs = 100)
w2vmodel.wv

In [ ]:
w2vmodel.wv.save_word2vec_format('word2vec.bin')

In [38]:
# 查看向量大小，内容
print(w2vmodel.wv["壮观"].shape)
w2vmodel.wv["壮观"]

(300,)


array([ 1.8548631e-03,  2.5859904e-03, -1.3251428e-03, -1.0319304e-03,
        3.2204110e-04, -3.0231769e-03, -3.2255924e-03,  3.0987498e-03,
       -2.9931942e-03,  1.0770377e-03,  1.7962956e-03, -3.2818571e-03,
        3.1451413e-03, -1.8959482e-04,  2.9692252e-05, -2.9860954e-03,
        2.8812743e-03, -2.9740731e-05,  1.6362993e-03,  9.0006192e-04,
        2.0483586e-03, -2.3376800e-03,  8.7025721e-04, -1.8786892e-03,
       -3.1604799e-03,  1.1875998e-03, -1.8168449e-04,  3.3838273e-04,
        5.6048471e-04,  2.1856432e-03,  1.3187035e-03, -1.5038173e-03,
       -1.3739292e-03, -1.6506815e-03,  3.2962681e-04, -1.3631423e-03,
       -1.8825547e-03, -1.8353966e-03,  1.5544113e-03,  1.8326362e-04,
        4.7706882e-04,  5.6970917e-04, -2.5846895e-03, -2.8141299e-03,
        1.3361669e-03,  1.6656180e-03,  2.2966259e-03, -2.3531718e-03,
        1.6218424e-03, -5.9313019e-04, -3.0291493e-03,  1.9100845e-03,
        3.1906276e-03, -4.9369771e-04,  1.6550906e-03,  1.0605932e-03,
      

In [59]:
# 寻找最相似的词语
print("爬山最相似的词：", w2vmodel.wv.most_similar("爬山", topn=10))
print("古镇最相似的词：", w2vmodel.wv.most_similar("古镇", topn=10))
print("西安最相似的词：", w2vmodel.wv.most_similar("西安", topn=10))
print("城墙最相似的词：", w2vmodel.wv.most_similar("城墙", topn=10))

爬山最相似的词： [('卖', 0.20316290855407715), ('找', 0.18993660807609558), ('吃', 0.1748582124710083), ('城墙', 0.16768117249011993), ('听', 0.1647062748670578), ('历史悠久', 0.14634107053279877), ('算', 0.14612026512622833), ('得名', 0.14136599004268646), ('完', 0.14109739661216736), ('景观', 0.1374412328004837)]
古镇最相似的词： [('当年', 0.17481419444084167), ('半小时', 0.14458531141281128), ('11', 0.141275554895401), ('昭陵', 0.14090465009212494), ('圣地', 0.13918648660182953), ('设施', 0.13244794309139252), ('时期', 0.12836028635501862), ('秦岭', 0.12740257382392883), ('大唐', 0.12212132662534714), ('西安', 0.12162892520427704)]
西安最相似的词： [('三个', 0.1968320906162262), ('里', 0.16358694434165955), ('算是', 0.15931393206119537), ('票', 0.15918275713920593), ('点', 0.14623825252056122), ('超级', 0.1433369517326355), ('5', 0.14186017215251923), ('一路', 0.14096122980117798), ('找', 0.13987284898757935), ('到达', 0.1393241435289383)]
城墙最相似的词： [('爬山', 0.16768115758895874), ('独特', 0.15255802869796753), ('之间', 0.14940521121025085), ('游人', 0.1490216255

In [72]:
index2word_set

{'1',
 '10',
 '11',
 '15',
 '2',
 '20',
 '3',
 '30',
 '4',
 '5',
 '6',
 '8',
 '一体',
 '一圈',
 '一处',
 '一座',
 '一游',
 '一点',
 '一种',
 '一路',
 '三',
 '三个',
 '上山',
 '下午',
 '下山',
 '不好',
 '不用',
 '不算',
 '不远',
 '世界',
 '东西',
 '两个',
 '中',
 '中华',
 '中国',
 '中央',
 '中心',
 '主题',
 '之地',
 '之间',
 '书法',
 '五丈',
 '交通',
 '享受',
 '人少',
 '人文',
 '介绍',
 '令人',
 '价格',
 '休闲',
 '众多',
 '优惠',
 '优美',
 '位于',
 '住',
 '体验',
 '佛像',
 '佛教',
 '便利',
 '便宜',
 '保存',
 '修建',
 '修路',
 '值得',
 '值得一看',
 '做',
 '停车',
 '停车场',
 '免费',
 '党',
 '全国',
 '八路军',
 '公交',
 '公园',
 '公里',
 '兰花',
 '关中',
 '兵马俑',
 '内容',
 '军委',
 '农家乐',
 '冬天',
 '凉快',
 '几个',
 '凤凰山',
 '出',
 '分',
 '分钟',
 '到达',
 '前',
 '前往',
 '动物',
 '北台',
 '千年',
 '半小时',
 '华夏',
 '华山',
 '单位',
 '卖',
 '博物馆',
 '历史',
 '历史悠久',
 '原',
 '原始',
 '去过',
 '县城',
 '发展',
 '发现',
 '古',
 '古代',
 '古迹',
 '古镇',
 '台',
 '吃',
 '听',
 '吸引',
 '呈',
 '周',
 '周末',
 '周边',
 '味道',
 '唯一',
 '善寺',
 '喜欢',
 '四季',
 '园',
 '国家',
 '国家级',
 '圣地',
 '均',
 '坐',
 '城',
 '城内',
 '城墙',
 '城市',
 '基地',
 '塔',
 '塬',
 '境内',
 '墓',
 '壁画',
 '壮观',
 '处',
 '夏天',
 '大兴',
 '大唐

In [74]:
import numpy as np
from scipy import spatial

index2word_set = set(w2vmodel.wv.index_to_key)

def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = jieba.cut(sentence)
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [83]:
view.txt[31]

'很美多不错的表演好看'